# 4. Heatmap (countour plot) of model
This notebook focuses on visualizing the predictions of a machine learning model in the form of a heatmap, or contour plot. The objective is to create a heatmap that represents how the model's predicted probability changes across two variables: starting glucose levels and the duration of exercise. Such a visualization can provide valuable insights into how these two factors potentially influence the risk of an outcome, as predicted by the model.

Outline:

1. Load Packages and Data: In this section, we load the necessary packages and the trained models.
2. Create Meshgrid: The meshgrid is a way to generate a 2D grid of coordinates, which we use for our x (starting glucose) and y (duration of exercise) values. This allows us to evaluate the model's predictions across a grid of combinations of starting glucose levels and exercise durations.
3. Get Model Predictions: Here, we evaluate the trained models on our grid of data points, obtaining a prediction for each point.
4. Reformat Data for Contour Plot: Before plotting, we need to organize the predictions in a way that's compatible with contour plotting.
5. Plot Heatmap: Finally, we plot the heatmap (contour plot) using Matplotlib. We visualize the results both in mg/dL and mmol/L units for glucose levels, providing a comprehensive view of how the predicted probabilities vary across different starting glucose concentrations and exercise durations.

## 4.0. Load packages and data

In [63]:
import matplotlib.pyplot as plt
from matplotlib import colors, gridspec
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
import pickle
import ml_helper as ml_help
import os
from numpy import argmax


In [48]:
directory = '../../data/tidy_data/final_df/'
# Read data
df = pd.read_csv(directory + 'ho_df.csv')
strat = df['stratify'] 
X = pd.read_csv(directory + 'ho_X.csv')
y = df['y'] 

In [49]:
def unpickle(i):
    with open(i, "rb") as fp:   
        #Unpickling 
        return pickle.load(fp)

In [136]:
directory = '../../results/models/xgb/'
models_two = []
for filename in os.listdir(directory):
    if filename.startswith('two_'):
        with open(directory + filename, 'rb') as file:
            models_two.append(pickle.load(file))

In [137]:
models_all = []
for filename in os.listdir(directory):
    if filename.startswith('all_'):
        with open(directory + filename, 'rb') as file:
            models_all.append(pickle.load(file))

In [138]:
# Convert to dataframe
test_df = X[['start_glc', 'duration']]

## 4.2. Get model predictions

In [139]:
# Empty list for predictions
predictions = []
for model in models_two:
    y_probas = model.predict_proba(test_df)
    y_probas = y_probas[:, 1].flatten()
    predictions.append(y_probas)

In [140]:
# Take the mean of the 10 model's predictions 
predicted_proba_two = np.mean(predictions, axis=0)

In [141]:
# Empty list for predictions
predictions_all = []
for model in models_all:
    y_probas = model.predict_proba(X)
    y_probas = y_probas[:, 1].flatten()
    predictions_all.append(y_probas)

In [142]:
# Take the mean of the 10 model's predictions 
predicted_proba_all = np.mean(predictions_all, axis=0)

In [143]:
roc_auc_score(y, predicted_proba_all)

0.8852290650331952

In [86]:
overall_results = []

# Create lists for k-fold results
threshold_accuracy = []
threshold_precision = []
threshold_recall = []
threshold_f1 = []
threshold_predicted_positive_rate = []
threshold_observed_positive_rate = []
threshold_true_positive_rate = []
threshold_false_positive_rate = []
threshold_specificity = []
threshold_balanced_accuracy = []

# Set up thresholds
thresholds = np.arange(0, 1.01, 0.01)
# Loop through increments in probability of survival
for cutoff in thresholds: #  loop 0 --> 1 on steps of 0.1
    # Get whether passengers survive using cutoff
    predicted_survived = predicted_proba >= cutoff
    # Call accuracy measures function
    accuracy = ml_help.calculate_accuracy(y, predicted_survived)
    # Add accuracy scores to lists
    threshold_accuracy.append(accuracy['accuracy'])
    threshold_precision.append(accuracy['precision'])
    threshold_recall.append(accuracy['recall'])
    threshold_f1.append(accuracy['f1'])
    threshold_predicted_positive_rate.append(
            accuracy['predicted_positive_rate'])
    threshold_observed_positive_rate.append(
            accuracy['observed_positive_rate'])
    threshold_true_positive_rate.append(accuracy['true_positive_rate'])
    threshold_false_positive_rate.append(accuracy['false_positive_rate'])
    threshold_specificity.append(accuracy['specificity'])
    threshold_balanced_accuracy.append((accuracy['specificity']+accuracy['recall'])/2)

# Select threshold with the best balance between sensitivity and specificity
ix = argmax(threshold_balanced_accuracy)
roc_auc = roc_auc_score(y, predicted_proba)

overall_results.append([roc_auc, thresholds[ix], threshold_accuracy[ix], threshold_precision[ix],
                threshold_recall[ix], threshold_f1[ix], threshold_predicted_positive_rate[ix],
                threshold_observed_positive_rate[ix], threshold_true_positive_rate[ix], 
                threshold_false_positive_rate[ix], threshold_specificity[ix], 
                threshold_balanced_accuracy[ix]])



In [134]:
predictions

[]

In [156]:
def calculate_metrics(predictions, y):
        overall_results = []
        for probs in predictions: 
                # Create lists for k-fold results
                threshold_accuracy = []
                threshold_precision = []
                threshold_recall = []
                threshold_f1 = []
                threshold_predicted_positive_rate = []
                threshold_observed_positive_rate = []
                threshold_true_positive_rate = []
                threshold_false_positive_rate = []
                threshold_specificity = []
                threshold_balanced_accuracy = []

                # Set up thresholds
                thresholds = np.arange(0, 1.01, 0.01)
                # Loop through increments in probability of survival
                for cutoff in thresholds: #  loop 0 --> 1 on steps of 0.1
                        # Get whether passengers survive using cutoff
                        predicted_survived = probs >= cutoff
                        # Call accuracy measures function
                        accuracy = ml_help.calculate_accuracy(y, predicted_survived)
                        # Add accuracy scores to lists
                        threshold_accuracy.append(accuracy['accuracy'])
                        threshold_precision.append(accuracy['precision'])
                        threshold_recall.append(accuracy['recall'])
                        threshold_f1.append(accuracy['f1'])
                        threshold_predicted_positive_rate.append(
                                accuracy['predicted_positive_rate'])
                        threshold_observed_positive_rate.append(
                                accuracy['observed_positive_rate'])
                        threshold_true_positive_rate.append(accuracy['true_positive_rate'])
                        threshold_false_positive_rate.append(accuracy['false_positive_rate'])
                        threshold_specificity.append(accuracy['specificity'])
                        threshold_balanced_accuracy.append((accuracy['specificity']+accuracy['recall'])/2)

                # Select threshold with the best balance between sensitivity and specificity
                ix = argmax(threshold_balanced_accuracy)
                roc_auc = roc_auc_score(y, probs)

                overall_results.append([roc_auc, thresholds[ix], threshold_accuracy[ix], threshold_precision[ix],
                                threshold_recall[ix], threshold_f1[ix], threshold_predicted_positive_rate[ix],
                                threshold_observed_positive_rate[ix], threshold_true_positive_rate[ix], 
                                threshold_false_positive_rate[ix], threshold_specificity[ix], 
                                threshold_balanced_accuracy[ix]])
        overall_results = pd.DataFrame(overall_results)
        overall_results.columns=['roc_auc', 'threshold','accuracy', 'precision', 'recall', 'f1','predicted_positive_rate', 'observed_positive_rate', 'tpr','fpr','specificity','balanced_accuracy']
        return overall_results

In [157]:
metrics_simple = calculate_metrics(predictions, y)
metrics_all = calculate_metrics(predictions_all, y)

In [165]:
simple_results_xgb = ml_help.calculate_confidence_intervals(metrics_simple, ['roc_auc','balanced_accuracy', 'recall', 'specificity']) #lr_two_k_fold_results
all_results_xgb = ml_help.calculate_confidence_intervals(metrics_all, ['roc_auc','balanced_accuracy', 'recall', 'specificity']) #lr_two_k_fold_results


In [172]:
all_results_xgb['two'] = simple_results_xgb['value']

In [174]:
all_results_xgb.T#.pivot_table(columns='metric')

,0,1,2,3
metric,roc_auc,balanced_accuracy,recall,specificity
value,"0.88 (0.87, 0.88)","0.79 (0.79, 0.80)","0.79 (0.77, 0.82)","0.79 (0.76, 0.82)"
two,"0.86 (0.86, 0.86)","0.77 (0.77, 0.77)","0.74 (0.72, 0.76)","0.79 (0.77, 0.82)"
